In [1]:
# Instalação das bibliotecas
!pip install transformers torch sentencepiece pandas accelerate bitsandbytes

# Atualização do transformers para garantir compatibilidade
!pip install --upgrade transformers

# Necessário para usar o Phi-3
!pip install 'optimum[onnxruntime]'

import sys
import os
import re
import json
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Garante a codificação UTF-8
try:
    sys.stdout.reconfigure(encoding='utf-8')
except Exception:
    pass

print("Setup de bibliotecas concluído.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 137.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# CÉLULA 2: CARREGAMENTO DOS DADOS LOCAIS DO FAQUAD-NLI (CSV)

# Importações essenciais
import sys
import os
import json
import torch
import pandas as pd
# pandas é necessário para ler o CSV

# =========================================================================
# 1. CARREGAMENTO DOS DADOS DO CSV
# =========================================================================

# O arquivo agora é CSV, carregado diretamente
DATA_FILE_PATH = "faquad_nli_parte.csv"
data_lines = [] # Inicializa a lista de todos os exemplos

try:
    print(f"Iniciando o carregamento do arquivo CSV: {DATA_FILE_PATH}...")

    # 1. Carrega o CSV para um DataFrame
    df = pd.read_csv(DATA_FILE_PATH)

    # 2. Converte o DataFrame para uma lista de dicionários para fácil iteração
    data_lines = df.to_dict('records')

    if not data_lines:
        raise ValueError("O DataFrame carregado está vazio.")

    print(f"\n✅ SUCESSO! {len(data_lines)} exemplos carregados do arquivo CSV.")

    # Exibe a primeira linha para confirmação das chaves
    example_row = data_lines[0]

    # Usamos as chaves reais do CSV inspecionado
    context_substituto = example_row.get('document_title', 'ERRO CONTEXTO').strip()
    expected_answer = example_row.get('answer', 'ERRO RESPOSTA').strip()

    print(f"Exemplo de contexto (document_title): {context_substituto[:50]}...")
    print(f"Resposta de referência ('answer'): {expected_answer}")

    print("\n⚠️ ALERTA: O CONTEXTO COMPLETO ESTÁ AUSENTE NO CSV. Usaremos 'document_title' como substituto.")

except Exception as e:
    print(f"\n❌ ERRO FATAL: Falha ao carregar o dataset do CSV.")
    print(f"Erro: {e}")
    sys.exit()


# =========================================================================
# VARIÁVEIS GLOBAIS DE DADOS
# =========================================================================

TARGET_INFERENCE = "pronominal bridging"
# 'data_lines' agora contém a lista completa de exemplos

Iniciando o carregamento do arquivo CSV: faquad_nli_parte.csv...

✅ SUCESSO! 100 exemplos carregados do arquivo CSV.
Exemplo de contexto (document_title): CIENCIA_DA_COMPUTACAO...
Resposta de referência ('answer'): A primeira ferramenta conhecida para a computação foi o ábaco, cuja invenção é atribuída a habitantes da Mesopotâmia, em torno de 2700–2300 a.C..

⚠️ ALERTA: O CONTEXTO COMPLETO ESTÁ AUSENTE NO CSV. Usaremos 'document_title' como substituto.


In [3]:
# CÉLULA 3: Carregamento dos Modelos (Phi-3 e BERT)

# Importações essenciais (Repetidas por segurança do escopo)
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# =========================================================================
# 3. CARREGAMENTO DE MODELOS (PHI-3 e BERT)
# =========================================================================

# Ajuste para usar a GPU (se disponível) ou CPU.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Usamos float16 para economia de VRAM (essencial para o Phi-3)
MODEL_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"\nCarregando gerador de perguntas (PHI-3-mini-4k-instruct) para dispositivo: {device}...")
qg_model_id = "microsoft/Phi-3-mini-4k-instruct"

try:
    # Phi-3 usa o pipeline 'text-generation'
    qg = pipeline(
        "text-generation",
        model=qg_model_id,
        device=device,
        model_kwargs={"torch_dtype": MODEL_DTYPE}
    )
except Exception as e:
    print(f"❌ ERRO ao carregar o modelo Phi-3: {e}")
    sys.exit()


print("Carregando classificador de skills (Modelo BERT)...")
cls_model_id = "curious008/BertForStorySkillClassification"
cls_tokenizer = AutoTokenizer.from_pretrained(cls_model_id, use_fast=False)
cls_model = AutoModelForSequenceClassification.from_pretrained(
    cls_model_id, ignore_mismatched_sizes=True
)
if torch.cuda.is_available():
    cls_model.to("cuda")

skill_labels = [
    "Character", "Setting", "Feeling", "Action",
    "Causal Relationship", "Outcome Resolution", "Prediction"
]

print("Carregamento de modelos concluído.")


Carregando gerador de perguntas (PHI-3-mini-4k-instruct) para dispositivo: cuda:0...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at curious008/BertForStorySkillClassification and are newly initialized because the shapes did not match:
- bert.embeddings.word_embeddings.weight: found shape torch.Size([30524, 768]) in the checkpoint and torch.Size([30523, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Carregamento de modelos concluído.


In [13]:
# CÉLULA 4: Geração Controlada (QG), Classificação e Salvamento - VERSÃO FINAL EM PORTUGUÊS

# Importações necessárias
from tqdm.notebook import tqdm
import re
import os
import json
import torch
import sys

# =========================================================================
# RESTAURAÇÃO DE VARIÁVEIS GLOBAIS
# =========================================================================
# (Assumidas da Célula 3)
cls_model_id = "curious008/BertForStorySkillClassification"
TARGET_INFERENCE = "pronominal bridging"
skill_labels = [
    "Character", "Setting", "Feeling", "Action",
    "Causal Relationship", "Outcome Resolution", "Prediction"
]

# =========================================================================
# FLUXO DO PROMPT (TRADUZIDO PARA PORTUGUÊS)
# =========================================================================

FEW_SHOT_EXAMPLE_TEMPLATE = f"""
[INSTRUÇÃO]
TAREFA: Gere uma pergunta de múltipla escolha para o tipo de inferência 'pronominal bridging'.
REGRAS: A saída deve aderir estritamente ao formato: NOVO_TIPO, RACIOCÍNIO, PERGUNTA, OPÇÕES. Todo o texto gerado (RACIOCÍNIO, PERGUNTA, OPÇÕES) DEVE estar em PORTUGUÊS.

Exemplo:
TIPO: pronominal bridging
CONTEXTO: Uma estufa é uma construção onde plantas como flores e vegetais são cultivados. Ela geralmente tem um teto de vidro ou plástico translúcido.
RESPOSTA: estufas
RACIOCÍNIO: O pronome 'Ela' se refere à 'estufa' na frase anterior, fazendo a ligação entre os conceitos.
PERGUNTA: De acordo com a passagem, o que pode ter tetos de plástico translúcido?
OPÇÕES: quintais; salas de estar; estufas; jardins botânicos

NOVO_CONTEXTO: {{context}}
NOVA_RESPOSTA: {{answer}}
NOVO_TIPO: pronominal bridging
RACIOCÍNIO: (Gere o raciocínio para a nova pergunta AQUI, em PORTUGUÊS)
PERGUNTA: (Gere o tema da nova pergunta AQUI, em PORTUGUÊS)
OPÇÕES: (Gere 4 opções, separadas por ponto e vírgula, em PORTUGUÊS)
"""

def create_phi3_prompt_for_inference_type(context, answer, template):
    """Cria o prompt Few-Shot com formatação de chat."""
    prompt = template.replace("{{context}}", context) \
                     .replace("{{answer}}", answer)
    return f"<|user|>{prompt}<|end|><|assistant|>"

# =========================================================================
# 4. LOOP PRINCIPAL DE GERAÇÃO E AVALIAÇÃO
# =========================================================================

# Variáveis assumidas de outras células: data_lines, qg, cls_tokenizer, cls_model

# **AJUSTE AQUI:** Limite o número de exemplos para testes.
MAX_EXAMPLES_TO_PROCESS = 5
all_results = []

print("\n=== INICIANDO PROCESSAMENTO EM LOTE (QG PHI-3 em PORTUGUÊS) ===")
print(f"Processando {min(MAX_EXAMPLES_TO_PROCESS, len(data_lines))} de {len(data_lines)} exemplos...")


for i, example_row in enumerate(tqdm(data_lines[:MAX_EXAMPLES_TO_PROCESS])):

    # 1. Extração de Contexto e Resposta
    context = example_row.get('passage', example_row.get('document_title', 'ERRO')).strip()
    expected_answer = example_row.get('answer', 'ERRO').strip()
    document_title = example_row.get('document_title', 'N/A')

    if context == 'ERRO' or expected_answer == 'ERRO':
        continue

    # 2. Geração Controlada (QG)
    qg_input = create_phi3_prompt_for_inference_type(context, expected_answer, FEW_SHOT_EXAMPLE_TEMPLATE)

    try:
        qg_output_list = qg(qg_input, max_new_tokens=512, do_sample=False, return_full_text=False)
        generated_output_full = qg_output_list[0]['generated_text'].strip()
    except Exception as e:
        generated_output_full = f"ERRO NA GERAÇÃO DO MODELO: {e}"

    # 3. Parsing (RQ3 e RQ1)
    reasoning = "N/A"
    generated_q = generated_output_full

    if generated_output_full != "ERRO NA GERAÇÃO DO MODELO":
        try:
            # Note: O Regex ainda procura por 'REASONING' e 'QUESTION' (em inglês) no código,
            # mesmo que o modelo deva gerar "RACIOCÍNIO" e "PERGUNTA".
            # Modelos como o Phi-3 geralmente mantêm as tags de cabeçalho em inglês
            # mesmo gerando o conteúdo em outro idioma.

            reasoning_match = re.search(r"RACIOCÍNIO:\s*(.*?)(?=\s*PERGUNTA:|$)", generated_output_full, re.DOTALL | re.IGNORECASE)
            question_match = re.search(r"PERGUNTA:\s*(.*?)(?=\s*OPÇÕES:|$)", generated_output_full, re.DOTALL | re.IGNORECASE)

            reasoning = reasoning_match.group(1).strip() if reasoning_match else "ERRO: Parsing do Raciocínio Falhou."
            generated_q = question_match.group(1).strip() if question_match else "ERRO: Parsing da Pergunta Falhou."

            # Limpeza e fallback
            generated_q = re.sub(r'\s+', ' ', generated_q).strip()
            if "ERRO" in generated_q:
                generated_q = re.sub(r'\s+', ' ', generated_output_full).strip()
                reasoning = "Parsing falhou; usando output completo como pergunta."
        except Exception:
            generated_q = re.sub(r'\s+', ' ', generated_output_full).strip()
            reasoning = "Parsing Geral Falhou."

    # 4. Classificação (RQ2)
    try:
        enc = cls_tokenizer(generated_q, return_tensors="pt", truncation=True, padding=True, max_length=512)
        if torch.cuda.is_available():
            enc = {k:v.to("cuda") for k,v in enc.items()}

        logits = cls_model(**enc).logits
        probs = torch.softmax(logits, dim=-1).cpu().detach().numpy().flatten()
        pred_idx = int(probs.argmax())
        pred_label = skill_labels[pred_idx]
    except Exception as e:
        pred_label = f"ERRO NA CLASSIFICAÇÃO: {str(e)[:50]}..."
        probs = [0.0] * len(skill_labels)

    # 5. Coleta de Resultados
    results_entry = {
        "id": i,
        "target_inference_type": TARGET_INFERENCE,
        "classification_model": cls_model_id,
        "document_title": document_title,
        "context_snippet": context,
        "expected_answer": expected_answer,
        "input_prompt_full": qg_input,
        "generated_question_stem_rq1": generated_q,
        "generated_reasoning_cot_rq3": reasoning,
        "predicted_skill_label_rq2": pred_label,
        "probabilities_by_class": {lbl: float(p) for lbl, p in zip(skill_labels, probs)}
    }
    all_results.append(results_entry)

print("\nProcessamento em lote concluído.")

# =========================================================================
# 5. SALVAMENTO DOS OUTPUTS
# =========================================================================

OUTPUT_DIR = "/content/output"
OUTPUT_FILENAME = "faquad_qg_classification_results.json"
outpath = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

os.makedirs(OUTPUT_DIR, exist_ok=True)

try:
    with open(outpath, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    print(f"\n✅ RESULTADOS FINAIS salvos com sucesso em: {outpath}")

except Exception as e:
    print(f"\n❌ ERRO ao salvar o arquivo JSON.")
    print(f"Erro: {e}")


=== INICIANDO PROCESSAMENTO EM LOTE (QG PHI-3 em PORTUGUÊS) ===
Processando 5 de 100 exemplos...


  0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Processamento em lote concluído.

✅ RESULTADOS FINAIS salvos com sucesso em: /content/output/faquad_qg_classification_results.json
